https://pythonprogramming.net/new-data-set-training-nltk-tutorial/

### Sentiment analysis on dataset with shorter texts

this one builds on the previous Notebooks

In [2]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize


In [3]:
class VoteClassifier(ClassifierI):
    # init method will always run in a class
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    # classify method
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    

In [4]:
# Load the data: positive and negative short texts

short_pos = open('data/positive.txt','r').read()
short_neg = open('data/negative.txt','r').read()


In [5]:
# Get the reviews in the list called documents
documents = []

for r in short_pos.split('\n'):
    documents.append((r, 'pos'))
    
for r in short_neg.split('\n'):
    documents.append((r, 'neg'))
    

In [6]:
# Get the words
all_words = []

short_pos_words = word_tokenize(short_pos)
short_neg_words = word_tokenize(short_neg)

for w in short_pos_words:
    all_words.append(w.lower())
    
for w in short_neg_words:
    all_words.append(w.lower())


In [7]:
# frequency distribution of words
all_words = nltk.FreqDist(all_words)


#### Convert words into features

In [8]:
# limit number of words (only 5,000 most common words)
word_features = list(all_words.keys())[:5000]


In [9]:
# Find the most common words (if they are present in a movie review)
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        
    return features


In [10]:
featuresets = [(find_features(rev), category) for (rev,category) in documents]

# shuffle the data
random.shuffle(featuresets)

# number of reviews
len(featuresets)


10662

#### Test different classifiers

In [11]:
# Train and test sets
training_set = featuresets[:10000]
testing_set = featuresets[10000:]


In [12]:
# Naive Bayes Classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

# Test the classifier
print('Original NB Classifier accuracy:', nltk.classify.accuracy(classifier, testing_set))


Original NB Classifier accuracy: 0.7522658610271903


In [13]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/naivebayes.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [14]:
classifier.show_most_informative_features(10)


Most Informative Features
                      `` = True              pos : neg    =     91.6 : 1.0
              engrossing = True              pos : neg    =     19.5 : 1.0
               inventive = True              pos : neg    =     14.9 : 1.0
               wonderful = True              pos : neg    =     12.9 : 1.0
              refreshing = True              pos : neg    =     12.9 : 1.0
                    warm = True              pos : neg    =     12.5 : 1.0
            refreshingly = True              pos : neg    =     11.6 : 1.0
                  stupid = True              neg : pos    =     11.5 : 1.0
                  beauty = True              pos : neg    =     11.3 : 1.0
             mesmerizing = True              pos : neg    =     10.9 : 1.0


In [15]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)

print('MNB_classifier accuracy:', nltk.classify.accuracy(MNB_classifier, testing_set))


MNB_classifier accuracy: 0.7537764350453172


In [16]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/MNB_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [17]:
# Classify one test example
print(MNB_classifier.classify_many(testing_set[0][0]))
print(MNB_classifier.classify(testing_set[0][0]))


['neg']
neg


In [18]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)

print('BernoulliNB_classifier accuracy:', nltk.classify.accuracy(BernoulliNB_classifier, testing_set))


BernoulliNB_classifier accuracy: 0.7583081570996979


In [19]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/BernoulliNB_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [20]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)

print('LogisticRegression_classifier accuracy:', (nltk.classify.accuracy(LogisticRegression_classifier, testing_set)))


C:\Users\Christoph\anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression_classifier accuracy: 0.7809667673716012


In [21]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/LogisticRegression_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [22]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)

print('SGDClassifier_classifier accuracy:', (nltk.classify.accuracy(SGDClassifier_classifier, testing_set)))


SGDClassifier_classifier accuracy: 0.7643504531722054


In [23]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/SGD_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [24]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)

print('SVC_classifier accuracy:', (nltk.classify.accuracy(SVC_classifier, testing_set)))


SVC_classifier accuracy: 0.770392749244713


In [25]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/SVCNB_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [26]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)

print('LinearSVC_classifier accuracy:', (nltk.classify.accuracy(LinearSVC_classifier, testing_set)))


LinearSVC_classifier accuracy: 0.7447129909365559


In [27]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/LinearSVC_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


In [28]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)

print('NuSVC_classifier accuracy:', (nltk.classify.accuracy(NuSVC_classifier, testing_set)))


NuSVC_classifier accuracy: 0.797583081570997


In [29]:
# Save the classifier with pickle
save_classifier = open('pickled_algorithms/NuSVC_classifier.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()


#### Combine algorithms for a voting system

https://pythonprogramming.net/combine-classifier-algorithms-nltk-tutorial/

it is best to have an odd number of classifiers, e.g., 4/7 vs. 3/7

In [30]:
voted_classifier = VoteClassifier(classifier,
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print('voted_classifier accuracy:', (nltk.classify.accuracy(voted_classifier, testing_set)))


voted_classifier accuracy: 0.7870090634441088


In [31]:
print('Classification:', voted_classifier.classify(testing_set[0][0]), 'Confidence:',voted_classifier.confidence(testing_set[0][0]))
print('Classification:', voted_classifier.classify(testing_set[1][0]), 'Confidence:',voted_classifier.confidence(testing_set[1][0]))
print('Classification:', voted_classifier.classify(testing_set[2][0]), 'Confidence:',voted_classifier.confidence(testing_set[2][0]))
print('Classification:', voted_classifier.classify(testing_set[3][0]), 'Confidence:',voted_classifier.confidence(testing_set[3][0]))
print('Classification:', voted_classifier.classify(testing_set[4][0]), 'Confidence:',voted_classifier.confidence(testing_set[4][0]))
print('Classification:', voted_classifier.classify(testing_set[5][0]), 'Confidence:',voted_classifier.confidence(testing_set[5][0]))


Classification: neg Confidence: 1.0
Classification: neg Confidence: 1.0
Classification: pos Confidence: 1.0
Classification: neg Confidence: 0.8571428571428571
Classification: pos Confidence: 1.0
Classification: neg Confidence: 1.0
